<a href="https://colab.research.google.com/github/SooinJung/NLP-/blob/main/NLP/word2vec_fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec vs FastText

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")


('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7da377fb4490>)

## 전처리

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [5]:
print('총 샘플의 개수 : {}'.format(len(result)))


총 샘플의 개수 : 273424


In [6]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec - Training

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원. - 에러가 떠서 실습이 되지 않는다.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.



In [8]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, window=5, min_count=5, workers=4, sg=0)

In [9]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8537586331367493), ('guy', 0.7937188744544983), ('lady', 0.7720308303833008), ('girl', 0.7678096294403076), ('boy', 0.756765604019165), ('gentleman', 0.7225412130355835), ('kid', 0.7119853496551514), ('soldier', 0.7033891081809998), ('poet', 0.6908872127532959), ('friend', 0.6514883637428284)]


In [10]:
# 모델 저장
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [11]:
model.wv.most_similar("woman")

[('man', 0.853758692741394),
 ('girl', 0.8467100858688354),
 ('lady', 0.8015455603599548),
 ('boy', 0.7801832556724548),
 ('soldier', 0.7332619428634644),
 ('kid', 0.7327806949615479),
 ('child', 0.7298033833503723),
 ('gentleman', 0.721863865852356),
 ('guy', 0.7066795229911804),
 ('poet', 0.6995641589164734)]

In [ ]:
# Word2Vec는 학습 데이터에 존재하지 않는 단어. 즉, 모르는 단어에 대해서는 임베딩 벡터가 존재하지 않기 때문에 단어의 유사도를 계산할 수 없다
# electrofishing은 새로 만들어낸 없는 단어.. 전기장어인가
model.wv.most_similar("electrofishing")

KeyError: ignored

## fastText - Training

In [12]:
from gensim.models import FastText
model = FastText(result, window=5, min_count=5, workers=4, sg=1)

In [13]:
# 한번 비교용..
model.wv.most_similar("woman")

[('girl', 0.8179001212120056),
 ('man', 0.8046596646308899),
 ('hoffman', 0.7726520895957947),
 ('womanhood', 0.7723931074142456),
 ('batman', 0.7680457830429077),
 ('lehman', 0.7672613859176636),
 ('ekman', 0.7601702213287354),
 ('boy', 0.7584890723228455),
 ('child', 0.7578121423721313),
 ('baptist', 0.7553804516792297)]

In [14]:
# 하지만 FastText은 electrofishing 단어를 쪼개서 벡터로 사용하기 때문에 유추가 가능해진다.
model.wv.most_similar("electrofishing")

[('electrolyte', 0.8603428602218628),
 ('electrolux', 0.8603293895721436),
 ('electro', 0.8523483872413635),
 ('electroshock', 0.8403630256652832),
 ('electroencephalogram', 0.8387399315834045),
 ('electric', 0.830112636089325),
 ('electrochemical', 0.8244895339012146),
 ('electrogram', 0.818213164806366),
 ('electrons', 0.816283643245697),
 ('electron', 0.8147240281105042)]